In [2]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import skimage.io as io
import pickle

import time

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer
from keras.regularizers import l2
from keras import backend as K

from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

## Loading data
a function that loads the images locations and labels.<br>
input: the data path.<br>
output:<br>
&emsp;&emsp;dataHiero -> a dataframe with index= location of images and label= their labels <br>
&emsp;&emsp;img_groups -> a dictionary in the shape of { "label" : [array of locations of images labeled with this label] }

In [ ]:
path="../GlyphDataset/Dataset/Manual/Preprocessed/"

def loadData(folderPictures=path):
    
    folders=next(os.walk(folderPictures))[1]
    img_groups = {}
    img_list={}

    for folder in folders:
        for img_file in os.listdir(folderPictures+folder):
            name, 
            label = img_file.strip('.png').split("_")
            
            
            # One image per class

            #if label not in img_groups.keys():
            #    img_groups[label] = [folder + "_" + name]


            # Multiple images per class

            if label in img_groups.keys():
                img_groups[label].append(folder+"_"+name)
            else:
                img_groups[label] = [folder+"_"+name]

            img_list[folder+"_"+name]=[label]


    # Remove class with only one hieroglyph


    for k,v in list(img_groups.items()):
        if len(v)==1: del img_groups[k]

    # Extract only N hieroglyph classes randomly

    nclass = len(img_groups.keys())

    list_of_class = random.sample(list(img_groups.keys()), nclass)
#     print(list_of_class)

    short_dico = {x: img_groups[x] for x in list_of_class if x in img_groups}

    dataHiero=pd.DataFrame.from_dict(img_list,orient='index')
    dataHiero.columns = ["label"]
    dataHiero = dataHiero[dataHiero.label != 'UNKNOWN']

    dataHiero = dataHiero.loc[dataHiero['label'].isin(short_dico)]


    dataHiero.reset_index(level=0, inplace=True)

    return dataHiero,img_groups

a function that takes the image groups and load those images<br>
input: img_proups dictionary<br>
output:<br>
&emsp;&emsp;X -> np array of the images<br>
&emsp;&emsp;y -> np array of labels<br>
&emsp;&emsp;glyph_sizes -> a dictionary in the form of {'label' : (starting index, ending index in X and y)}


In [ ]:
def read_images(img_groups,path):
    X=[]
    y=[]
    glyph_sizes={}
    low=0
    for glyph in img_groups:
        category_images=[]
        high=low
        for img_path in img_groups[glyph] :
            folder,name = img_path.split('_')
            image = io.imread(path+folder+'/'+name+'_'+glyph+'.png')
            X.append(image)
            y.append(glyph)
            high+=1
#         X.append(np.array(category_images))
        glyph_sizes[glyph]=(low,high-1)
        low=high
        
    return np.array(X),np.array(y).reshape((-1,1)),glyph_sizes
            
    

In [ ]:
dataHiero,img_groups=loadData(folderPictures=path)
dataHiero.head()
# img_groups

,index,label
0,39_390115,D21
1,39_390082,M17
2,39_390292,V31
3,39_390375,U15
4,39_390175,D35


In [ ]:
X,y,sizes=read_images(img_groups,path)

In [ ]:
type(X)
print(y.shape)
print(X.shape)
sizes['D21'][1]

(2921, 1)
(2921, 75, 50)


5

saving the images into a pickle

In [ ]:
#train val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state=42)

In [ ]:
def get_sizes(X,Y):
    sizes={}
    for i ,(x,y) in enumerate(zip(X,Y)):
#         print(i,x,y)
        if y[0] in sizes:
            sizes[y[0]].append(i)
        else:
            sizes[y[0]]=[i]
    return sizes
    

In [ ]:
sizes=get_sizes(X_train,y_train)
X=X_train
y=y_train

In [ ]:
#saving data as pickle
with open("train.pickle", "wb") as f:
    pickle.dump((X,y,sizes),f)

In [ ]:
sizes_val=get_sizes(X_test,y_test)
Xval=X_test
yval=y_test

In [ ]:
#saving data as pickle
with open("test.pickle", "wb") as f:
    pickle.dump((Xval,yval,sizes_val),f)

In [ ]:
# #saving data as pickle
# with open("test.pickle", "wb") as f:
#     pickle.dump((Xval,yval,sizes_val),f)

## reading the training tensors

In [50]:
#colab
data_path= '/content/drive/MyDrive/hiero_cv/'
#local
# data_path='./'

In [51]:
with open(data_path+"train.pickle", "rb") as f:
    (X,y,sizes) = pickle.load(f)

In [13]:
with open(data_path+"test.pickle", "rb") as f:
    (Xval,yval,sizes_val) = pickle.load(f)

In [11]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
# def get_siamese_model_2(input_shape):
#     """
#         Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
#     """
    
#     # Define the tensors for the two input images
#     left_input = Input(input_shape)
#     right_input = Input(input_shape)
    
#     # Convolutional Neural Network
#     model = Sequential()
#     model.add(Conv2D(64, (3,3),strides=(2, 2), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
#     model.add(MaxPooling2D((2, 2), strides=2))
#     model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
#     model.add(MaxPooling2D((2, 2), strides=2))
#     model.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
# #     model.add(MaxPooling2D())
# #     model.add(Conv2D(256, (4,4), activation='relu',  kernel_regularizer=l2(2e-4)))
#     model.add(Flatten())
#     model.add(Dense(4096, activation='sigmoid',
#                    kernel_regularizer=l2(1e-3)))
    
#     # Generate the encodings (feature vectors) for the two images
#     encoded_l = model(left_input)
#     encoded_r = model(right_input)
    
#     # Add a customized layer to compute the absolute difference between the encodings
#     L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#     L1_distance = L1_layer([encoded_l, encoded_r])
    
#     # Add a dense layer with a sigmoid unit to generate the similarity score
#     prediction = Dense(1,activation='sigmoid')(L1_distance)
    
#     # Connect the inputs with the outputs
#     siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
#     # return the model
#     return siamese_net

In [ ]:
# model_2 = get_siamese_model_2((75, 50, 1))
# model_2.summary()

In [ ]:
# optimizer = Adam(lr = 0.001)
# model_2.compile(loss="binary_crossentropy",optimizer=optimizer)

In [ ]:
# def transfer_model(input_shape):
#     """
#         Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
#     """
    
#     # Define the tensors for the two input images
#     left_input = Input(input_shape)
#     right_input = Input(input_shape)
    
#     #Import inception model for transfer learning without output layers
#     base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = input_shape)
    
    
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     # let's add a fully-connected layer
#     x = Dense(1024, activation='relu')(x)
#     # and a logistic layer -- let's say we have 200 classes
#     model= Dense(200, activation='softmax')(x)
    
#     # this is the model we will train
# #     model = Model(inputs=base_model.input, outputs=predictions)
        
    
#     # Generate the encodings (feature vectors) for the two images
#     encoded_l = model(left_input)
#     encoded_r = model(right_input)
    
#     # Add a customized layer to compute the absolute difference between the encodings
#     L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#     L1_distance = L1_layer([encoded_l, encoded_r])
    
#     # Add a dense layer with a sigmoid unit to generate the similarity score
#     prediction = Dense(1,activation='sigmoid')(L1_distance)
    
#     # Connect the inputs with the outputs
#     model = Model(inputs=[left_input,right_input],outputs=prediction)
    
#     for layer in model.layers[:249]:
#        layer.trainable = False
#     for layer in model.layers[249:]:
#        layer.trainable = True
    
#     # return the model
#     return model

In [ ]:
# inception_model = transfer_model((75, 50, 1))
# inception_model.summary()

In [ ]:
# inception_model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [14]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (3,3), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu',  kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3)))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [15]:
model = get_siamese_model((75, 50, 1))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 75, 50, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 75, 50, 1)]  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         4891712     input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           sequential_1[0][0]         

In [16]:
optimizer = Adam(lr = 0.0001)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


a function that create pairs of images with y= 1 if they are similar and 0 if they are different. 

## a function to predict which glyph

In [17]:
def create_glyphlist(X,sizes):
    images=[]
    labels=[]
    _,w,h=X.shape
    for glyph in sizes:
        index=sizes[glyph][0]
        images.append(X[index].reshape( w , h, 1))
        labels.append(glyph)
    return np.asarray(images), np.asarray(labels) 

In [18]:
anchor_img, anchor_label=create_glyphlist(X,sizes)

In [19]:
def whichGlyph_pair(image,anchor_img,anchor_label):
    N,w,h,_=anchor_img.shape
#     pairs=[np.zeros((N, w, h,1)) for i in range(2)]
    
    test_image= np.asarray([image]*N).reshape(N, w, h,1)
    
    anchor_label, test_image, anchor_img = shuffle(anchor_label, test_image, anchor_img)
#     pairs = [test_image,anchor_img]
    
    return test_image, anchor_img, anchor_label
    

In [20]:
def whichGlyph(model,image,anchor_img,anchor_label):
    test_image,anchor_img,targets = whichGlyph_pair(image,anchor_img,anchor_label)
    probs = model.predict([test_image,anchor_img])
    return probs,anchor_img,targets
    

In [21]:
# def whichGlyph(model,image,anchor_img,anchor_label):
#     test_image,anchor_img,targets = whichGlyph_pair(image,anchor_img,anchor_label)
#     probs=[]
#     for i in range(0,len(targets),2):
#         pair=[[test_image[i],test_image[i+1]],[anchor_img[i],anchor_img[i+1]]]
#         pred=model.predict(pair)
#         for p in pred:
#             probs.append(p)
        
#     return probs,anchor_img,targets

## creating pairs of images

In [22]:
def createPairs(X,y,sizes,batch_size):
    ##create a batch with half it's size are similar glyphs and the other half are different.
    n=0
    i=0
    
    label=[]
    _,w,h=X.shape
    # initialize 2 empty arrays for the input image batch
#     pairs=[np.zeros((batch_size, w, h,1)) for i in range(2)]
    input1=np.zeros((batch_size, w, h,1))
    input2=np.zeros((batch_size, w, h,1))
    
    while n < batch_size:
        random_key1=random.choice(list(sizes))
#         low=sizes[random_key1][0]
#         high=sizes[random_key1][1]
        index1, index3 = np.random.choice(sizes[random_key1], size=2)
        index2 = np.random.choice(sizes[random_key1])
        random_key2=random.choice(list(sizes))
        
        while random_key2 == random_key1:
            random_key2=random.choice(list(sizes))
            
#         low=sizes[random_key2][0]
#         high=sizes[random_key2][1]
        index4=np.random.choice(sizes[random_key2])
        n += 2
        # appending images 1 and 3 into input1 and input2 corresponding to y=1 
        #and images 2 and 4 corresponding to y=0
    
        input1[i,:,:,:] = X[index1].reshape( w , h, 1)
        input1[i+1,:,:,:] = X[index2].reshape(w, h, 1)
        input2[i,:,:,:] = X[index3].reshape(w, h, 1)
        input2[i+1,:,:,:] = X[index4].reshape(w, h, 1)
        i += 2
#         input1+=[X[index1],X[index2]]
#         input2+=[X[index3],X[index4]]
        label+=[1,0]
        
#         print(index1,index2,index3,index4)
#         print(y[index1],y[index2],y[index3],y[index4])
#         print(random_key1,random_key2)
    input1,input2,label = shuffle(input1,input2,label)
    pairs=[input1,input2]
    
    return pairs,label
pairs,label=createPairs(X,y,sizes,32)   

In [23]:
# pairs[1].shape
# label

a fn that creates a N-way one shot learning task where it create pairs with the wanted image and N-1 different ones and 1 similar one.

In [33]:
def make_oneshot_task(X,y,sizes,N):
    _,w,h=X.shape
    pairs=[np.zeros((N, w, h,1)) for i in range(2)]
    
    true_key=random.choice(list(sizes))
#     low=sizes[true_key][0]
#     high=sizes[true_key][1]
    
    index=np.random.choice(sizes[true_key])
    index2=np.random.choice(sizes[true_key])
    
    test_image= np.asarray([X[index]]*N).reshape(N, w, h,1)
#     print(X.shape)
    X_diff= np.delete(X,sizes[true_key],axis=0)
#     print(X_diff.shape)
    indices= np.random.choice(range(0, len(X_diff)), size = N-1)
    
    support_set=X_diff[indices,:,:]
    ##!!! adding the similar image to the start of the array is not working!!!!!!!!!!!!!!!!!!!!!
    
#     print(support_set.shape)
#     support_set = [ X[index2] ] + support_set
    support_set=np.insert(support_set,0,X[index2],axis=0)
#     print(support_set.shape)
    targets = np.zeros((N,))
    targets[0] = 1
    support_set=support_set.reshape(N,w,h,1)
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets
    


In [34]:
pp,tt=make_oneshot_task(Xval,yval,sizes_val,20) 
pp[0].shape

(20, 75, 50, 1)

In [26]:
# plt.imshow(pairs[0][0])
pairs[1].shape
# len(label)

(32, 75, 50, 1)

In [27]:
def test_oneshot(model,X,y,sizes, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(X,y,sizes,N)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [28]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations 20000
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [38]:
model_path = './weights/'
model_2_path= '/content/drive/MyDrive/hiero_cv/weights/'

In [39]:
X.shape

(2921, 75, 50)

In [40]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = createPairs(X,y,sizes,batch_size)
    targets=np.asarray(targets)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model,Xval,yval,sizes_val, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_2_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------

 ------------- 

Time for 200 iterations: 0.15213268200556437 mins
Train Loss: 0.7059872150421143
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 32.0% 20 way one-shot learning accuracy 

Current best: 32.0, previous best: -1

 ------------- 

Time for 400 iterations: 0.5822814504305521 mins
Train Loss: 0.5833199620246887
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 36.0% 20 way one-shot learning accuracy 

Current best: 36.0, previous best: 32.0

 ------------- 

Time for 600 iterations: 0.9915398955345154 mins
Train Loss: 0.6169869303703308
Evaluating model on 250 random 20 way one-shot learning tasks ... 

Got an average of 48.4% 20 way one-shot learning accuracy 

Current best: 48.4, previous best: 36.0

 ------------- 

Time for 800 iterations: 1.4218302249908448 mins
Train Loss: 0.5006300210952759
Evaluating model on 250 random 20 way on

In [ ]:
# print("Starting training process!")
# print("-------------------------------------")
# t_start = time.time()
# for i in range(1, n_iter+1):
#     (inputs,targets) = createPairs(X,y,sizes,batch_size)
#     loss = model.train_on_batch(inputs, targets)
#     if i % evaluate_every == 0:
#         print("\n ------------- \n")
#         print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
#         print("Train Loss: {0}".format(loss)) 
#         val_acc = test_oneshot(model,Xval,yval,sizes_val, N_way, n_val, verbose=True)
#         model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
#         if val_acc >= best:
#             print("Current best: {0}, previous best: {1}".format(val_acc, best))
#             best = val_acc

## loading model from weights

In [ ]:
# model_path=

In [ ]:
model.load_weights(model_path+'weights.20000.h5')

In [43]:
def calc_accuracy(N,Xval,yval,anchor_img,anchor_label,model):
    count_first=0
    count_first3=0
    for i in range(N):
        ind=random.choice(range(yval.shape[0]))
        predicted,anchor_imgs,targets=whichGlyph(model,Xval[ind],anchor_img,anchor_label)
        sort_index = np.argsort(np.asarray(predicted).reshape(len(predicted),))
        if targets[sort_index[-1]] == yval[ind][0]:
            count_first+=1
        if yval[ind][0] in targets[sort_index[127:]]:
            count_first3+=1
    accuracy_first=count_first/N
    accuracy_first3=count_first3/N
    
    return accuracy_first, accuracy_first3

In [44]:
t_start = time.time()
acc1,acc3=calc_accuracy(250,Xval,yval,anchor_img,anchor_label,model)
print(f'testing:\nfound first accuracy = {acc1} , first 3 accuracy = {acc3} ')
print("accuracy fn took {0} mins".format((time.time()-t_start)/60.0))

testing:
found first accuracy = 0.688 , first 3 accuracy = 0.972 
accuracy fn took 0.4506264050801595 mins


In [45]:
t_start = time.time()
acc1,acc3=calc_accuracy(250,X,y,anchor_img,anchor_label,model)
print(f'training:\nfound first accuracy = {acc1} , first 3 accuracy = {acc3} ')
print("accuracy fn took {0} mins".format((time.time()-t_start)/60.0))

training:
found first accuracy = 0.82 , first 3 accuracy = 0.992 
accuracy fn took 0.447417942682902 mins


In [46]:
# t_start = time.time()
# print(calc_accuracy(1,Xval,yval,anchor_img,anchor_label,model))
# print("accuracy fn took {0} sec".format((time.time()-t_start)))

(1.0, 1.0)
accuracy fn took 0.10917329788208008 sec
